In [169]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import regularizers
from sklearn.metrics import accuracy_score
import numpy as np
from IPython.display import clear_output

clear_output()

In [170]:
df = pd.read_pickle('data/saeb_pb_filtered.pkl')
df.head()

,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,amostra,proficiencia,erro_padrao,proficiencia_saeb,erro_padrao_saeb,desempenho_aluno,sexo,raca_cor,ano_nascimento,idade,possui_eletrecidade,possui_escrivaninha,escolaridade_mae,escolaridade_pai,responsaveis_leem,responsaveis_incentivam_realizacao_licao_casa,responsaveis_incentivam_estudos,responsaveis_incentivam_comparecer_aulas,tempo_chegada_escola,forma_chegada_escola,transporte_escolar,tempo_lazer,tempo_cursos,tempo_estudos,leitura_noticias,leitura_livros_geral,leitura_historia_quadrinhos
0,2021,PB,1,49369547,1,CH,0,-1.076008,0.479786,189.482456,28.124102,None,.,.,<NA>,<NA>,.,.,.,.,.,.,.,.,.,.,.,.,.,.,None,None,None
1,2021,PB,0,48935836,1,CH,1,0.212093,0.483460,264.988420,28.339465,None,A,A,2006,15,B,B,E,D,B,C,C,C,B,C,B,D,C,B,None,None,None
2,2021,PB,1,49370798,1,CH,0,-1.093155,0.665270,188.477352,38.996806,None,.,.,<NA>,<NA>,.,.,.,.,.,.,.,.,.,.,.,.,.,.,None,None,None
3,2021,PB,1,49163192,1,CH,0,-2.013929,0.610384,134.503387,35.779498,None,B,C,2005,16,.,.,F,F,.,.,C,.,C,F,B,.,.,A,None,None,None
4,2021,PB,1,49355938,1,CH,0,-1.622314,0.677914,157.459089,39.737972,None,.,.,<NA>,<NA>,.,.,.,.,.,.,.,.,.,.,.,.,.,.,None,None,None


In [171]:
# selecting categorical columns
columns_to_filter = ['possui_eletrecidade', 'possui_escrivaninha', 'escolaridade_mae', 'escolaridade_pai', 'responsaveis_leem', 'responsaveis_incentivam_realizacao_licao_casa', 'responsaveis_incentivam_estudos', 'responsaveis_incentivam_comparecer_aulas', 'tempo_chegada_escola', 'forma_chegada_escola', 'transporte_escolar', 'tempo_lazer', 'tempo_cursos', 'tempo_estudos']

# remove rows with all values '.' or '*'
df = df[(df[columns_to_filter] != '.').all(axis=1)]
df = df[(df[columns_to_filter] != '*').all(axis=1)]
print(len(df))

df.head()

118745


,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,amostra,proficiencia,erro_padrao,proficiencia_saeb,erro_padrao_saeb,desempenho_aluno,sexo,raca_cor,ano_nascimento,idade,possui_eletrecidade,possui_escrivaninha,escolaridade_mae,escolaridade_pai,responsaveis_leem,responsaveis_incentivam_realizacao_licao_casa,responsaveis_incentivam_estudos,responsaveis_incentivam_comparecer_aulas,tempo_chegada_escola,forma_chegada_escola,transporte_escolar,tempo_lazer,tempo_cursos,tempo_estudos,leitura_noticias,leitura_livros_geral,leitura_historia_quadrinhos
1,2021,PB,0,48935836,1,CH,1,0.212093,0.483460,264.988420,28.339465,None,A,A,2006,15,B,B,E,D,B,C,C,C,B,C,B,D,C,B,None,None,None
17,2021,PB,0,48935834,1,CH,1,-1.302525,0.700937,176.204509,41.087535,None,A,C,2007,14,B,B,E,C,A,C,B,C,A,F,B,C,A,B,None,None,None
63,2021,PB,1,49164452,1,CH,0,-0.207503,0.511880,240.392529,30.005389,None,A,A,2006,15,B,B,F,F,B,C,C,C,C,A,B,A,C,C,None,None,None
64,2021,PB,1,49368109,1,CH,0,-0.765889,0.611475,207.661042,35.843450,None,B,C,2005,16,B,A,F,F,A,C,C,C,A,A,A,D,C,D,None,None,None
65,2021,PB,1,49353247,1,CH,0,-0.686009,0.559540,212.343416,32.799123,None,B,C,2005,16,A,A,A,F,B,C,C,C,A,D,B,A,A,C,None,None,None


In [172]:
# print columns with dtype object
print(df.select_dtypes(include=['object']).columns)

Index(['sigla_uf', 'id_aluno', 'disciplina', 'desempenho_aluno', 'sexo',
       'raca_cor', 'possui_eletrecidade', 'possui_escrivaninha',
       'escolaridade_mae', 'escolaridade_pai', 'responsaveis_leem',
       'responsaveis_incentivam_realizacao_licao_casa',
       'responsaveis_incentivam_estudos',
       'responsaveis_incentivam_comparecer_aulas', 'tempo_chegada_escola',
       'forma_chegada_escola', 'transporte_escolar', 'tempo_lazer',
       'tempo_cursos', 'tempo_estudos', 'leitura_noticias',
       'leitura_livros_geral', 'leitura_historia_quadrinhos'],
      dtype='object')


In [173]:
# nulling '2021' from 'idade'
df['idade'] = df['idade'].apply(lambda x: x if x != 2021 else np.nan)


# change 'idade' type to float (temporarily)
df['idade'] = df['idade'].astype(float)

# fill 'idade' missing values with the mean (int)
df.fillna({'idade': df['idade'].mean()}, inplace=True)
df['idade'] = df['idade'].astype(int)

df['idade'].head()

1     15
17    14
63    15
64    16
65    16
Name: idade, dtype: int64

In [174]:
print(df['idade'].value_counts())

print('\n')

print(df['idade'].describe())

idade
14    83147
15    25475
16     7224
17     2899
Name: count, dtype: int64


count    118745.000000
mean         14.409449
std           0.714114
min          14.000000
25%          14.000000
50%          14.000000
75%          15.000000
max          17.000000
Name: idade, dtype: float64


In [175]:
df['desempenho_aluno'].value_counts(dropna=False)

desempenho_aluno
Básico          60283
Insuficiente    26051
Proficiente     21071
None             7636
Avançado         3704
Name: count, dtype: int64

In [176]:
print(df['proficiencia_saeb'].isnull().sum())

# drop null values from 'proficiencia_saeb'
df.dropna(subset=['proficiencia_saeb'], inplace=True)

print(df['proficiencia_saeb'].isnull().sum())

2
0


In [177]:
# changing 'desempenho_aluno' to categorical numerical values
# Insuficiente = 0, Básico = 1, Proficiente = 2, Avançado = 3
# dropping None values

# drop the ones with value 'None'
_ = df.dropna(subset=['desempenho_aluno']).copy()

_.loc[:, 'desempenho_aluno_cat'] = _['desempenho_aluno'].map({'Insuficiente': 0, 'Básico': 1, 'Proficiente': 2, 'Avançado': 3})
#df['desempenho_aluno'] = df['desempenho_aluno'].astype(int)

df = _

df['desempenho_aluno_cat'].value_counts(dropna=False)

desempenho_aluno_cat
1    60283
0    26051
2    21071
3     3704
Name: count, dtype: int64

In [179]:
# change categorical columns (with dtype = 'object') to numerical values
# using LabelEncoder
le = LabelEncoder()

for column in columns_to_filter:
    df[column] = le.fit_transform(df[column])

df.head()

,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,amostra,proficiencia,erro_padrao,proficiencia_saeb,erro_padrao_saeb,desempenho_aluno,sexo,raca_cor,ano_nascimento,idade,possui_eletrecidade,possui_escrivaninha,escolaridade_mae,escolaridade_pai,responsaveis_leem,responsaveis_incentivam_realizacao_licao_casa,responsaveis_incentivam_estudos,responsaveis_incentivam_comparecer_aulas,tempo_chegada_escola,forma_chegada_escola,transporte_escolar,tempo_lazer,tempo_cursos,tempo_estudos,leitura_noticias,leitura_livros_geral,leitura_historia_quadrinhos,desempenho_aluno_cat
69,2021,PB,1,43575034,1,MT,0,-0.220310,0.350673,237.650742,19.599930,Básico,A,C,<NA>,14,1,0,2,0,0,2,2,2,0,0,0,2,2,1,None,None,None,1
70,2021,PB,1,43575034,1,LP,0,-1.053385,0.447439,191.950468,24.650919,Insuficiente,A,C,<NA>,14,1,0,2,0,0,2,2,2,0,0,0,2,2,1,None,None,None,0
71,2021,PB,1,49370848,1,LP,0,-0.785038,0.389307,206.734607,21.448232,Básico,A,A,2006,15,1,0,0,3,2,2,2,2,0,3,1,0,0,2,None,None,None,1
126,2021,PB,1,43517493,1,MT,0,-0.357253,0.377772,229.996679,21.114556,Básico,A,A,2004,17,1,1,5,5,1,2,2,2,0,0,0,3,0,2,None,None,None,1
127,2021,PB,1,43509591,1,MT,0,1.000746,0.363297,305.898405,20.305515,Proficiente,B,A,2006,15,1,0,2,0,1,2,2,2,0,3,1,2,0,3,None,None,None,2


In [181]:
save = True

if save:
    df.to_pickle('data/saeb_treated.pkl')